## 如何派生内置不可变类型并修改实例化行为?

我们想自定义一种新类型的元组，对于传入的可迭代对象，我们只保留作其中`int`类型且值大于0的元素

In [9]:
class IntTuple(tuple):
    
    def __new__(cls, iterable):
        g = (x for x in iterable if isinstance(x, int) and x > 0)
        return super(IntTuple, cls).__new__(cls, g)
    
    def __init__(self, iterable):
        # before
        super(IntTuple, self).__init__()
        # after
        

t = IntTuple([1, -1, 'abc', 6, ['x', 'y'], 3])
print(t)

(1, 6, 3)


## 如何为创建大量实例节省内存?

某网络游戏中，定义了玩家类Player(id, name, status, ...)  
每有一个在线玩家，在服务器程序内则有一个Player的实例，当在线人数很多时，将产生大量实例。（如百万级）

如何降低这些大量实例的内存开销?

In [10]:
class Player(object):
    
    def __init__(self, uid, name, status=0, level=1):
        self.uid = uid
        self.name = name
        self.stat = status
        self.level = level
        
class Player2(object):
    
    __slots__ = ['uid', 'name', 'stat', 'level']
    
    def __init__(self, uid, name, status=0, level=1):
        self.uid = uid
        self.name = name
        self.stat = status
        self.level = level

In [11]:
p1 = Player('0001', 'Jim')
p2 = Player2('0001', 'Jim')

In [14]:
print(dir(p1))
print()
print(dir(p2))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'level', 'name', 'stat', 'uid']

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', 'level', 'name', 'stat', 'uid']


In [15]:
set(dir(p1)) - set(dir(p2))

{'__dict__', '__weakref__'}

In [19]:
p1.__dict__['x'] = 123

In [20]:
p1.x

123

In [22]:
del p1.__dict__['x']

In [23]:
p1.x

AttributeError: 'Player' object has no attribute 'x'

In [24]:
import sys

sys.getsizeof(p1.__dict__)

368

## 如何让对象支持上下文管理?

In [27]:
with open('demo.txt', 'w') as f:
    f.write('abcdef')
    f.writelines(['\nxyz', '\n123'])

我们实现了一个telnet客户端的类TelnetClient，调用实例的`start()`方法启动客户端与服务器交互，
交互完毕后需调用`cleanup()`方法，关闭已连接的socket，以及将操作历史纪录写入文件并关闭。

能否让TelnetClient的实例支持上下文管理协议，从而替代手工调用`cleanup()`方法。

实现上下文管理协议，需定义实例的 `__enter__`,`__exit__`方法，它们分别在with开始和结束时被调用。

In [ ]:
from telnetlib import Telnet
from sys import stdin, stdout
from collections import deque

class TelnetClient(object):

    def __init__(self, addr, port=23):
        self.addr = addr
        self.port = port
        self.tn = None

    def start(self):
        self.tn = Telnet(self.addr, self.port)
        self.history = deque()

        # user
        t = self.tn.read_until('login: ')
        stdout.write(t)
        user = stdin.readline()
        self.tn.write(user)

        # password
        t = self.tn.read_until('Password: ')
        if t.startswith(user[:-1]): t = t[len(user) + 1:]
        stdout.write(t)
        self.tn.write(stdin.readline())

        t = self.tn.read_until('$ ')
        stdout.write(t)
        while True:
            uinput = stdin.readline()
            if not uinput:
                break
            self.history.append(uinput)
            self.sn.write(uinput)
            t = self.tn.read_until('$ ')
            stdout.write(t[len(uinput) + 1:])

        def cleanup(self):
            self.tn.close()
            self.tn = None
            with open(self.addr + '_history.txt', 'w') as f:
                f.writelines(self.history)


client = TelnetClient('127.0.0.1')
print('\nstart...')
client.start()
print('\ncleanup')
client.cleanup() 

In [ ]:
from telnetlib import Telnet
from sys import stdin, stdout
from collections import deque

class TelnetClient(object):

    def __init__(self, addr, port=23):
        self.addr = addr
        self.port = port
        self.tn = None

    def start(self):
        # user
        t = self.tn.read_until('login: ')
        stdout.write(t)
        user = stdin.readline()
        self.tn.write(user)

        # password
        t = self.tn.read_until('Password: ')
        if t.startswith(user[:-1]): t = t[len(user) + 1:]
        stdout.write(t)
        self.tn.write(stdin.readline())

        t = self.tn.read_until('$ ')
        stdout.write(t)
        while True:
            uinput = stdin.readline()
            if not uinput:
                break
            self.history.append(uinput)
            self.sn.write(uinput)
            t = self.tn.read_until('$ ')
            stdout.write(t[len(uinput) + 1:])

        def cleanup(self):
            pass
                
        def __enter__(self):
            self.tn = Telnet(self.addr, self.port)
            self.history = deque()
            return self
        
        def __exit__(self, exc_type, exc_val, exc_tb):
            self.tn.close()
            self.tn = None
            with open(self.addr + '_history.txt', 'w') as f:
                f.writelines(self.history)
            

with TelnetClient('127.0.0.1') as client:
    client.start()

'''
client = TelnetClient('127.0.0.1')
print('\nstart...')
client.start()
print('\ncleanup')
client.cleanup() 
'''

## 如何创建可管理的对象属性?

In [5]:
from math import pi
    
class Circle(object):
    
    def __init__(self, radius):
        self.radius = radius
        
    def getRadius(self):
        return self.radius
    
    def setRadius(self, value):
        if not isinstance(value, (int, float)):
            raise ValueError('wrong type.')
        self.radius = float(value)
        
    def getArea(self):
        return self.radius ** 2 * pi
    
c = Circle(3.2)
c.getRadius()

3.2

使用`property`函数为类创建可管理属性, fget/fset/fdel对应相应属性访问.

In [9]:
from math import pi
    
class Circle(object):
    
    def __init__(self, radius):
        self.radius = radius
        
    def getRadius(self):
        return self.radius
    
    def setRadius(self, value):
        if not isinstance(value, (int, float)):
            raise ValueError('wrong type.')
        self.radius = float(value)
        
    def getArea(self):
        return self.radius ** 2 * pi
    
    R = property(getRadius, setRadius)
    
c = Circle(3.2)
print(c.R)
c.R = 5.9
print(c.R)
c.R = 'abc'

3.2
5.9


ValueError: wrong type.

## 如何让类支持比较操作?

比较符号运算符重载, 需要实现一下方法:  
`__lt__`, `__le__`, `__gt__`, `__ge__`, `__eq__`, `__ne__`

In [15]:
class Rectangle(object):
    
    def __init__(self, w, h):
        self.w = w
        self.h = h
        
    def area(self):
        return self.w * self.h
    
    def __lt__(self, obj):
        print('in __lt__')
        return self.area() < obj.area()
    
    def __le__(self, obj):
        print('in __le__')
        return self.area() <= obj.area()
    
    
r1 = Rectangle(2, 8)
r2 = Rectangle(4, 4)

print(r1 <= r2) # r1.__lt__(r2)

in __le__
True


使用标准库下的 functools 下的类装饰器`total_ordering`可以简化此过程

In [25]:
from functools import total_ordering
from abc import ABCMeta, abstractmethod

@total_ordering
class Shape(object):
    @abstractmethod
    def area(self):  # 抽象方法
        pass
    
    def __lt__(self, obj):
        print('in __lt__')
        if not isinstance(obj, Shape):
            raise TypeError('obj is not Shape')
        return self.area() < obj.area()
    
    def __eq__(self, obj):
        print('in __le__')
        if not isinstance(obj, Shape):
            raise TypeError('obj is not Shape')
        return self.area() == obj.area()

    
class Rectangle(Shape):
    
    def __init__(self, w, h):
        self.w = w
        self.h = h
        
    def area(self):
        return self.w * self.h
    
    
class Circle(Shape):
    
    def __init__(self, r):
        self.r = r
                
    def area(self):
        return self.r ** 2 * 3.1415926

r1 = Rectangle(3, 5)
r2 = Rectangle(4, 4)
c1 = Circle(3)

print(r1 > r2)
print()
print(r1 >= c1)
print()
print(c1 > r2)

in __lt__
False

in __lt__
False

in __lt__
in __le__
True


## 如何使用描述符对实例属性做类型检查?

In [28]:
class Descriptor(object):
    
    def __get__(self, instance, cls):
        print('in __get__', instance, cls)
        # return instance.__dict__[xxx]
        
    def __set__(self, instance, value):
        print('in __set__')
        # instance.__dict__[xxx] = value
        
    def __delete__(self, instance):
        print('in __del__')
        # del instance.__dict__[xxx]
        
    
class A(object):
    x = Descriptor()
    
a = A()
a.x
A.x

in __get__ <__main__.A object at 0x0000016E81C09048> <class '__main__.A'>
in __get__ None <class '__main__.A'>


In [30]:
a.x = 5
del a.x

in __set__
in __del__


In [31]:
a.x
print(a.__dict__)

in __get__ <__main__.A object at 0x0000016E81C09048> <class '__main__.A'>
{}


In [32]:
class Descriptor(object):
    
    def __get__(self, instance, cls):
        print('in __get__', instance, cls)
        return instance.__dict__['x']
        
    def __set__(self, instance, value):
        print('in __set__')
        instance.__dict__['x'] = value
        
    def __delete__(self, instance):
        print('in __del__')
        # del instance.__dict__[xxx]
        
    
class A(object):
    x = Descriptor()
    
a = A()
a.x = 5
print(a.x)

in __set__
in __get__ <__main__.A object at 0x0000016E81BF9438> <class '__main__.A'>
5


In [36]:
class Attr(object):
    
    def __init__(self, name, type_):
        self.name = name
        self.type_ = type_
    
    def __get__(self, instance, cls):
        return instance.__dict__[self.name]
        
    def __set__(self, instance, value):
        if not isinstance(value, self.type_):
            raise TypeError('expected an %s' % self.type_)
        instance.__dict__[self.name] = value
        
    def __delete__(self, instance):
        del instance.__dict__[self.name]
        
    
class Person(object):
    name = Attr('name', str)
    age = Attr('age', int)
    height = Attr('height', float)
    
p = Person()
p.name = 'Bob'
print(p.name)
p.age = '17'

Bob


TypeError: expected an <class 'int'>

## 如何在环状数据结构中管理内存?

In [3]:
class A(object):
    
    def __del__(self):
        print('in A.__del__')

In [4]:
a = A()
import sys
sys.getrefcount(a)

2

In [5]:
sys.getrefcount?

In [6]:
sys.getrefcount(a) - 1

1

In [7]:
a2 = a
sys.getrefcount(a) - 1

2

In [8]:
del a2

In [9]:
sys.getrefcount(a) - 1

1

In [10]:
a = 5

in A.__del__


In [14]:
class Data(object):
    
    def __init__(self, value, owner):
        self.owner = owner
        self.value = value
        
    def __str__(self):
        return "%s's data, value is %s" % (self.owner, self.value)
    
    def __del__(self):
        print('in Data.__del__')
        

class Node(object):
    
    def __init__(self, value):
        self.data = Data(value, self)
        
    def __del__(self):
        print('in Node.__del__')


node = Node(100)
del node
# import gc
# gc.collect()
print('wait...')

wait...


使用标准库`weakref`, 它可以创建一种能访问对象但不增加引用计数的对象.

In [15]:
a = A()
sys.getrefcount(a) - 1

1

In [16]:
import weakref
a_wref = weakref.ref(a)

a2 = a_wref()

a is a2

True

In [17]:
sys.getrefcount(a) - 1

2

In [18]:
del a
del a2

in A.__del__


In [19]:
a_wref() is None

True

In [20]:
import weakref

class Data(object):
    
    def __init__(self, value, owner):
        self.owner = weakref.ref(owner)
        self.value = value
        
    def __str__(self):
        return "%s's data, value is %s" % (self.owner(), self.value)
    
    def __del__(self):
        print('in Data.__del__')
        

class Node(object):
    
    def __init__(self, value):
        self.data = Data(value, self)
        
    def __del__(self):
        print('in Node.__del__')


node = Node(100)
del node
# import gc
# gc.collect()
print('wait...')

in Node.__del__
in Data.__del__
wait...


## 如何通过实例方法名字的字符串调用方法?

方法一：使用内置函数`getattr`通过名字在实例上获取方法对象, 然后调用.  

In [ ]:
from lib1 import Circle
from lib2 import Triangle
from lib3 import Rectangle

def getArea(shape):
    for name in ('area', 'getArea', 'get_area'):
        f = getattr(shape, name, None)
        if f:
            return f()
        
shape1 = Circle(2)
shape2 = Triangle(3, 4, 5)
shape3 = Rectangle(6, 4)


shapes = [shape1, shape2, shape3]
print(map(getArea, shapes))

方法二：使用标准库`poerator`下的`methodcaller`函数调用

In [21]:
from operator import methodcaller
methodcaller?

in Node.__del__
in Data.__del__
in Node.__del__
in Data.__del__


In [22]:
s = 'abc123abc456'
s.find('abc', 4)

6

In [25]:
m = methodcaller('find', 'abc', 4)
m

operator.methodcaller('find', 'abc', 4)

In [27]:
m(s)

6